In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import Levenshtein
import numpy as np
import math
import statistics
from sklearn.metrics import confusion_matrix

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve
import numpy as np
from tqdm import tqdm

In [3]:
dataset1 = pd.read_csv("dataset1_25000.csv")
dataset1

,USER_ID,Thil_count,Thil_mean,Thil_median,Thil_pstdev,Thil_max,Thil_min,Thpl_count,Thpl_mean,Thpl_median,...,curl_median,curl_pstdev,curl_max,curl_min,cuhl_count,cuhl_mean,cuhl_median,cuhl_pstdev,cuhl_max,cuhl_min
0,21833_1,1.0,27889.000000,27889.0,0.000000,27889.0,27889.0,1.0,112225.000000,112225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21833_2,1.0,15129.000000,15129.0,0.000000,15129.0,15129.0,1.0,555025.000000,555025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,233209_2,3.0,3744.666667,3969.0,2135.730372,6241.0,1024.0,3.0,132320.333333,20736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39734_2,2.0,2592.500000,2592.5,1503.500000,4096.0,1089.0,2.0,23552.500000,23552.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,285679_2,2.0,9176.500000,9176.5,1432.500000,10609.0,7744.0,2.0,43840.000000,43840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42730,422325_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42731,446102_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42732,266602_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42733,40737_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
dataset2 = pd.read_csv("dataset2_25000.csv")
dataset2

,User ID #1,User ID #2,Label
0,21833_1,21833_2,0
1,233209_2,233209_1,0
2,39734_2,39734_1,0
3,285679_2,285679_1,0
4,206052_1,206052_2,0
...,...,...,...
42307,318919_2,13746_1,1
42308,43583_2,343077_1,1
42309,372199_2,190097_1,1
42310,319660_2,342567_1,1


In [5]:
import numpy as np
from tqdm import tqdm

def user_features_generator(dataset):
    for user_id, user_features in dataset.groupby("USER_ID"):
        yield user_id, user_features.drop(columns=["USER_ID"])

def process_dataset(dataset2, user_features_generator):
    user_features_list = list(user_features_generator) 
    X = []
    y = []
    total_iterations = len(dataset2)

    for _, row in tqdm(dataset2.iterrows(), total=total_iterations, desc="Processing rows"):
        user1, user2, label = row["User ID #1"], row["User ID #2"], row["Label"]
        user1_features = next((features for uid, features in user_features_list if uid == user1), None)
        user2_features = next((features for uid, features in user_features_list if uid == user2), None)
        
        if user1_features is not None and user2_features is not None:
            squared_diff = np.square(user1_features.values - user2_features.values)
            X.append(squared_diff.flatten())
            y.append(label)

    return np.array(X), np.array(y)

user_features_gen = user_features_generator(dataset1)
X, y = process_dataset(dataset2, user_features_gen)

print(X)
print(y)

Processing rows: 100%|██████████| 42312/42312 [01:30<00:00, 469.13it/s]


[[0.00000000e+00 1.62817600e+08 1.62817600e+08 ...            nan
             nan            nan]
 [           nan            nan            nan ...            nan
             nan            nan]
 [           nan            nan            nan ...            nan
             nan            nan]
 ...
 [           nan            nan            nan ...            nan
             nan            nan]
 [           nan            nan            nan ...            nan
             nan            nan]
 [9.00000000e+00 1.65875081e+08 1.61976529e+08 ...            nan
             nan            nan]]
[0 0 0 ... 1 1 1]


In [6]:
X

array([[0.00000000e+00, 1.62817600e+08, 1.62817600e+08, ...,
                   nan,            nan,            nan],
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       ...,
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       [9.00000000e+00, 1.65875081e+08, 1.61976529e+08, ...,
                   nan,            nan,            nan]])

In [7]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [8]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_test, X_validate, y_test, y_validate = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
eer_threshold_idx_test = np.argmin(np.abs(tpr_test - (1 - fpr_test)))
fpr_test_val = fpr_test[eer_threshold_idx_test]
tpr_test_val = tpr_test[eer_threshold_idx_test]
eer_test_val = (fpr_test_val + 1 - tpr_test_val) / 2
print("FPR for Test Set:", fpr_test_val)
print("TPR for Test Set:", tpr_test_val)
print("EER for Test Set:", eer_test_val)


y_pred_validate = model.predict(X_validate)
accuracy_validate = accuracy_score(y_validate, y_pred_validate)
print("Validation Accuracy:", accuracy_validate)
print("Validation Confusion Matrix:")
print(confusion_matrix(y_validate, y_pred_validate))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

fpr_val, tpr_val, _ = roc_curve(y_validate, model.predict_proba(X_validate)[:, 1])
eer_threshold_idx_val = np.argmin(np.abs(tpr_val - (1 - fpr_val)))
fpr_val_val = fpr_val[eer_threshold_idx_val]
tpr_val_val = tpr_val[eer_threshold_idx_val]
eer_val_val = (fpr_val_val + 1 - tpr_val_val) / 2
print("FPR for Validation Set:", fpr_val_val)
print("TPR for Validation Set:", tpr_val_val)
print("EER for Validation Set:", eer_val_val)

Accuracy: 0.6547551522026848
Confusion Matrix:
[[1544 1109]
 [ 717 1919]]
FPR for Test Set: 0.3750471164719186
TPR for Test Set: 0.6946130500758725
EER for Test Set: 0.3402170331980231
Validation Accuracy: 0.6592928719984874
Validation Confusion Matrix:
[[1550 1097]
 [ 705 1937]]
FPR for Validation Set: 0.376275028333963
TPR for Validation Set: 0.6915215745647237
EER for Validation Set: 0.34237672688461956


In [9]:
dataset1a = pd.read_csv("dataset1_25000.csv")
dataset1a

,USER_ID,Thil_count,Thil_mean,Thil_median,Thil_pstdev,Thil_max,Thil_min,Thpl_count,Thpl_mean,Thpl_median,...,curl_median,curl_pstdev,curl_max,curl_min,cuhl_count,cuhl_mean,cuhl_median,cuhl_pstdev,cuhl_max,cuhl_min
0,21833_1,1.0,27889.000000,27889.0,0.000000,27889.0,27889.0,1.0,112225.000000,112225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21833_2,1.0,15129.000000,15129.0,0.000000,15129.0,15129.0,1.0,555025.000000,555025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,233209_2,3.0,3744.666667,3969.0,2135.730372,6241.0,1024.0,3.0,132320.333333,20736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39734_2,2.0,2592.500000,2592.5,1503.500000,4096.0,1089.0,2.0,23552.500000,23552.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,285679_2,2.0,9176.500000,9176.5,1432.500000,10609.0,7744.0,2.0,43840.000000,43840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42730,422325_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42731,446102_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42732,266602_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42733,40737_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
dataset1a.fillna(0, inplace=True)

def user_features_generator(dataset):
    for user_id, user_features in dataset.groupby("USER_ID"):
        yield user_id, user_features.drop(columns=["USER_ID"])

def process_dataset(dataset2, user_features_generator):
    user_features_list = list(user_features_generator) 
    X = []
    y = []
    total_iterations = len(dataset2)

    for _, row in tqdm(dataset2.iterrows(), total=total_iterations, desc="Processing rows"):
        user1, user2, label = row["User ID #1"], row["User ID #2"], row["Label"]
        user1_features = next((features for uid, features in user_features_list if uid == user1), None)
        user2_features = next((features for uid, features in user_features_list if uid == user2), None)
        
        if user1_features is not None and user2_features is not None:
            squared_diff = np.square(user1_features.values - user2_features.values)
            X.append(squared_diff.flatten())
            y.append(label)

    return np.array(X), np.array(y)

user_features_gen = user_features_generator(dataset1a)
X, y = process_dataset(dataset2, user_features_gen)

Processing rows: 100%|██████████| 42312/42312 [01:19<00:00, 530.63it/s]


In [11]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_test, X_validate, y_test, y_validate = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
eer_threshold_idx_test = np.argmin(np.abs(tpr_test - (1 - fpr_test)))
fpr_test_val = fpr_test[eer_threshold_idx_test]
tpr_test_val = tpr_test[eer_threshold_idx_test]
eer_test_val = (fpr_test_val + 1 - tpr_test_val) / 2
print("FPR for Test Set:", fpr_test_val)
print("TPR for Test Set:", tpr_test_val)
print("EER for Test Set:", eer_test_val)


y_pred_validate = model.predict(X_validate)
accuracy_validate = accuracy_score(y_validate, y_pred_validate)
print("Validation Accuracy:", accuracy_validate)
print("Validation Confusion Matrix:")
print(confusion_matrix(y_validate, y_pred_validate))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

fpr_val, tpr_val, _ = roc_curve(y_validate, model.predict_proba(X_validate)[:, 1])
eer_threshold_idx_val = np.argmin(np.abs(tpr_val - (1 - fpr_val)))
fpr_val_val = fpr_val[eer_threshold_idx_val]
tpr_val_val = tpr_val[eer_threshold_idx_val]
eer_val_val = (fpr_val_val + 1 - tpr_val_val) / 2
print("FPR for Validation Set:", fpr_val_val)
print("TPR for Validation Set:", tpr_val_val)
print("EER for Validation Set:", eer_val_val)

Accuracy: 0.42352051427491016
Confusion Matrix:
[[1080 1573]
 [1476 1160]]
FPR for Test Set: 0.5770825480588013
TPR for Test Set: 0.42298937784522
EER for Test Set: 0.5770465851067906
Validation Accuracy: 0.4133106447343543
Validation Confusion Matrix:
[[1110 1537]
 [1566 1076]]
FPR for Validation Set: 0.5859463543634303
TPR for Validation Set: 0.4144587433762301
EER for Validation Set: 0.5857438054936002


In [12]:
dataset1b = pd.read_csv("dataset1_25000.csv")
dataset1b

,USER_ID,Thil_count,Thil_mean,Thil_median,Thil_pstdev,Thil_max,Thil_min,Thpl_count,Thpl_mean,Thpl_median,...,curl_median,curl_pstdev,curl_max,curl_min,cuhl_count,cuhl_mean,cuhl_median,cuhl_pstdev,cuhl_max,cuhl_min
0,21833_1,1.0,27889.000000,27889.0,0.000000,27889.0,27889.0,1.0,112225.000000,112225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21833_2,1.0,15129.000000,15129.0,0.000000,15129.0,15129.0,1.0,555025.000000,555025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,233209_2,3.0,3744.666667,3969.0,2135.730372,6241.0,1024.0,3.0,132320.333333,20736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39734_2,2.0,2592.500000,2592.5,1503.500000,4096.0,1089.0,2.0,23552.500000,23552.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,285679_2,2.0,9176.500000,9176.5,1432.500000,10609.0,7744.0,2.0,43840.000000,43840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42730,422325_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42731,446102_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42732,266602_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42733,40737_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
median_values = dataset1b.median()
dataset1b.fillna(median_values, inplace=True)

def user_features_generator(dataset):
    for user_id, user_features in dataset.groupby("USER_ID"):
        yield user_id, user_features.drop(columns=["USER_ID"])

def process_dataset(dataset2, user_features_generator):
    user_features_list = list(user_features_generator) 
    X = []
    y = []
    total_iterations = len(dataset2)

    for _, row in tqdm(dataset2.iterrows(), total=total_iterations, desc="Processing rows"):
        user1, user2, label = row["User ID #1"], row["User ID #2"], row["Label"]
        user1_features = next((features for uid, features in user_features_list if uid == user1), None)
        user2_features = next((features for uid, features in user_features_list if uid == user2), None)
        
        if user1_features is not None and user2_features is not None:
            squared_diff = np.square(user1_features.values - user2_features.values)
            X.append(squared_diff.flatten())
            y.append(label)

    return np.array(X), np.array(y)

user_features_gen = user_features_generator(dataset1b)
X, y = process_dataset(dataset2, user_features_gen)

Processing rows: 100%|██████████| 42312/42312 [01:18<00:00, 538.74it/s]


In [14]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_test, X_validate, y_test, y_validate = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
eer_threshold_idx_test = np.argmin(np.abs(tpr_test - (1 - fpr_test)))
fpr_test_val = fpr_test[eer_threshold_idx_test]
tpr_test_val = tpr_test[eer_threshold_idx_test]
eer_test_val = (fpr_test_val + 1 - tpr_test_val) / 2
print("FPR for Test Set:", fpr_test_val)
print("TPR for Test Set:", tpr_test_val)
print("EER for Test Set:", eer_test_val)


y_pred_validate = model.predict(X_validate)
accuracy_validate = accuracy_score(y_validate, y_pred_validate)
print("Validation Accuracy:", accuracy_validate)
print("Validation Confusion Matrix:")
print(confusion_matrix(y_validate, y_pred_validate))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

fpr_val, tpr_val, _ = roc_curve(y_validate, model.predict_proba(X_validate)[:, 1])
eer_threshold_idx_val = np.argmin(np.abs(tpr_val - (1 - fpr_val)))
fpr_val_val = fpr_val[eer_threshold_idx_val]
tpr_val_val = tpr_val[eer_threshold_idx_val]
eer_val_val = (fpr_val_val + 1 - tpr_val_val) / 2
print("FPR for Validation Set:", fpr_val_val)
print("TPR for Validation Set:", tpr_val_val)
print("EER for Validation Set:", eer_val_val)

Accuracy: 0.30686330119115146
Confusion Matrix:
[[ 822 1831]
 [1835  801]]
FPR for Test Set: 0.6946852619675838
TPR for Test Set: 0.30538694992412746
EER for Test Set: 0.6946491560217282
Validation Accuracy: 0.3129135942522216
Validation Confusion Matrix:
[[ 890 1757]
 [1877  765]]
FPR for Validation Set: 0.6898375519455988
TPR for Validation Set: 0.30999242997728993
EER for Validation Set: 0.6899225609841544


In [15]:
dataset1c = pd.read_csv("dataset1_25000.csv")
dataset1c

,USER_ID,Thil_count,Thil_mean,Thil_median,Thil_pstdev,Thil_max,Thil_min,Thpl_count,Thpl_mean,Thpl_median,...,curl_median,curl_pstdev,curl_max,curl_min,cuhl_count,cuhl_mean,cuhl_median,cuhl_pstdev,cuhl_max,cuhl_min
0,21833_1,1.0,27889.000000,27889.0,0.000000,27889.0,27889.0,1.0,112225.000000,112225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21833_2,1.0,15129.000000,15129.0,0.000000,15129.0,15129.0,1.0,555025.000000,555025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,233209_2,3.0,3744.666667,3969.0,2135.730372,6241.0,1024.0,3.0,132320.333333,20736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39734_2,2.0,2592.500000,2592.5,1503.500000,4096.0,1089.0,2.0,23552.500000,23552.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,285679_2,2.0,9176.500000,9176.5,1432.500000,10609.0,7744.0,2.0,43840.000000,43840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42730,422325_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42731,446102_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42732,266602_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42733,40737_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
mean_values = dataset1c.mean()
dataset1c.fillna(mean_values, inplace = True)

def user_features_generator(dataset):
    for user_id, user_features in dataset.groupby("USER_ID"):
        yield user_id, user_features.drop(columns=["USER_ID"])

def process_dataset(dataset2, user_features_generator):
    user_features_list = list(user_features_generator) 
    X = []
    y = []
    total_iterations = len(dataset2)

    for _, row in tqdm(dataset2.iterrows(), total=total_iterations, desc="Processing rows"):
        user1, user2, label = row["User ID #1"], row["User ID #2"], row["Label"]
        user1_features = next((features for uid, features in user_features_list if uid == user1), None)
        user2_features = next((features for uid, features in user_features_list if uid == user2), None)
        
        if user1_features is not None and user2_features is not None:
            squared_diff = np.square(user1_features.values - user2_features.values)
            X.append(squared_diff.flatten())
            y.append(label)

    return np.array(X), np.array(y)

user_features_gen = user_features_generator(dataset1c)
X, y = process_dataset(dataset2, user_features_gen)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_test, X_validate, y_test, y_validate = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
eer_threshold_idx_test = np.argmin(np.abs(tpr_test - (1 - fpr_test)))
fpr_test_val = fpr_test[eer_threshold_idx_test]
tpr_test_val = tpr_test[eer_threshold_idx_test]
eer_test_val = (fpr_test_val + 1 - tpr_test_val) / 2
print("FPR for Test Set:", fpr_test_val)
print("TPR for Test Set:", tpr_test_val)
print("EER for Test Set:", eer_test_val)


y_pred_validate = model.predict(X_validate)
accuracy_validate = accuracy_score(y_validate, y_pred_validate)
print("Validation Accuracy:", accuracy_validate)
print("Validation Confusion Matrix:")
print(confusion_matrix(y_validate, y_pred_validate))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

fpr_val, tpr_val, _ = roc_curve(y_validate, model.predict_proba(X_validate)[:, 1])
eer_threshold_idx_val = np.argmin(np.abs(tpr_val - (1 - fpr_val)))
fpr_val_val = fpr_val[eer_threshold_idx_val]
tpr_val_val = tpr_val[eer_threshold_idx_val]
eer_val_val = (fpr_val_val + 1 - tpr_val_val) / 2
print("FPR for Validation Set:", fpr_val_val)
print("TPR for Validation Set:", tpr_val_val)
print("EER for Validation Set:", eer_val_val)

Processing rows: 100%|██████████| 42312/42312 [01:16<00:00, 555.57it/s]


Accuracy: 0.4244658725657024
Confusion Matrix:
[[1150 1503]
 [1541 1095]]
FPR for Test Set: 0.5748209574067094
TPR for Test Set: 0.42526555386949927
EER for Test Set: 0.5747777017686051
Validation Accuracy: 0.4248440158820193
Validation Confusion Matrix:
[[1189 1458]
 [1584 1058]]
FPR for Validation Set: 0.5746127691726483
TPR for Validation Set: 0.4254352763058289
EER for Validation Set: 0.5745887464334097


In [17]:
dataset1d = pd.read_csv("dataset1_25000.csv")
dataset1d

,USER_ID,Thil_count,Thil_mean,Thil_median,Thil_pstdev,Thil_max,Thil_min,Thpl_count,Thpl_mean,Thpl_median,...,curl_median,curl_pstdev,curl_max,curl_min,cuhl_count,cuhl_mean,cuhl_median,cuhl_pstdev,cuhl_max,cuhl_min
0,21833_1,1.0,27889.000000,27889.0,0.000000,27889.0,27889.0,1.0,112225.000000,112225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21833_2,1.0,15129.000000,15129.0,0.000000,15129.0,15129.0,1.0,555025.000000,555025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,233209_2,3.0,3744.666667,3969.0,2135.730372,6241.0,1024.0,3.0,132320.333333,20736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39734_2,2.0,2592.500000,2592.5,1503.500000,4096.0,1089.0,2.0,23552.500000,23552.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,285679_2,2.0,9176.500000,9176.5,1432.500000,10609.0,7744.0,2.0,43840.000000,43840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42730,422325_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42731,446102_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42732,266602_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42733,40737_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
dataset1d = dataset1d.applymap(lambda x: np.log1p(x) if isinstance(x, (int, float)) else x)

def user_features_generator(dataset):
    for user_id, user_features in dataset.groupby("USER_ID"):
        yield user_id, user_features.drop(columns=["USER_ID"])

def process_dataset(dataset2, user_features_generator):
    user_features_list = list(user_features_generator) 
    X = []
    y = []
    total_iterations = len(dataset2)

    for _, row in tqdm(dataset2.iterrows(), total=total_iterations, desc="Processing rows"):
        user1, user2, label = row["User ID #1"], row["User ID #2"], row["Label"]
        user1_features = next((features for uid, features in user_features_list if uid == user1), None)
        user2_features = next((features for uid, features in user_features_list if uid == user2), None)
        
        if user1_features is not None and user2_features is not None:
            squared_diff = np.square(user1_features.values - user2_features.values)
            X.append(squared_diff.flatten())
            y.append(label)

    return np.array(X), np.array(y)

user_features_gen = user_features_generator(dataset1d)
X, y = process_dataset(dataset2, user_features_gen)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_test, X_validate, y_test, y_validate = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
eer_threshold_idx_test = np.argmin(np.abs(tpr_test - (1 - fpr_test)))
fpr_test_val = fpr_test[eer_threshold_idx_test]
tpr_test_val = tpr_test[eer_threshold_idx_test]
eer_test_val = (fpr_test_val + 1 - tpr_test_val) / 2
print("FPR for Test Set:", fpr_test_val)
print("TPR for Test Set:", tpr_test_val)
print("EER for Test Set:", eer_test_val)


y_pred_validate = model.predict(X_validate)
accuracy_validate = accuracy_score(y_validate, y_pred_validate)
print("Validation Accuracy:", accuracy_validate)
print("Validation Confusion Matrix:")
print(confusion_matrix(y_validate, y_pred_validate))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

fpr_val, tpr_val, _ = roc_curve(y_validate, model.predict_proba(X_validate)[:, 1])
eer_threshold_idx_val = np.argmin(np.abs(tpr_val - (1 - fpr_val)))
fpr_val_val = fpr_val[eer_threshold_idx_val]
tpr_val_val = tpr_val[eer_threshold_idx_val]
eer_val_val = (fpr_val_val + 1 - tpr_val_val) / 2
print("FPR for Validation Set:", fpr_val_val)
print("TPR for Validation Set:", tpr_val_val)
print("EER for Validation Set:", eer_val_val)

Processing rows: 100%|██████████| 42312/42312 [01:15<00:00, 558.73it/s]


Accuracy: 0.683494044242768
Confusion Matrix:
[[1677  976]
 [ 698 1938]]
FPR for Test Set: 0.33810780248774974
TPR for Test Set: 0.702959028831563
EER for Test Set: 0.31757438682809336
Validation Accuracy: 0.6816033276611836
Validation Confusion Matrix:
[[1666  981]
 [ 703 1939]]
FPR for Validation Set: 0.33169625991688706
TPR for Validation Set: 0.6922785768357305
EER for Validation Set: 0.3197088415405783


In [19]:
dataset1e = pd.read_csv("dataset1_25000.csv")
dataset1e

,USER_ID,Thil_count,Thil_mean,Thil_median,Thil_pstdev,Thil_max,Thil_min,Thpl_count,Thpl_mean,Thpl_median,...,curl_median,curl_pstdev,curl_max,curl_min,cuhl_count,cuhl_mean,cuhl_median,cuhl_pstdev,cuhl_max,cuhl_min
0,21833_1,1.0,27889.000000,27889.0,0.000000,27889.0,27889.0,1.0,112225.000000,112225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21833_2,1.0,15129.000000,15129.0,0.000000,15129.0,15129.0,1.0,555025.000000,555025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,233209_2,3.0,3744.666667,3969.0,2135.730372,6241.0,1024.0,3.0,132320.333333,20736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39734_2,2.0,2592.500000,2592.5,1503.500000,4096.0,1089.0,2.0,23552.500000,23552.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,285679_2,2.0,9176.500000,9176.5,1432.500000,10609.0,7744.0,2.0,43840.000000,43840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42730,422325_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42731,446102_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42732,266602_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42733,40737_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
dataset1e = dataset1e.applymap(lambda x: np.log1p(x) if isinstance(x, (int, float)) else x)
mean_values = dataset1e.mean()
dataset1e.fillna(mean_values, inplace = True)

def user_features_generator(dataset):
    for user_id, user_features in dataset.groupby("USER_ID"):
        yield user_id, user_features.drop(columns=["USER_ID"])

def process_dataset(dataset2, user_features_generator):
    user_features_list = list(user_features_generator) 
    X = []
    y = []
    total_iterations = len(dataset2)

    for _, row in tqdm(dataset2.iterrows(), total=total_iterations, desc="Processing rows"):
        user1, user2, label = row["User ID #1"], row["User ID #2"], row["Label"]
        user1_features = next((features for uid, features in user_features_list if uid == user1), None)
        user2_features = next((features for uid, features in user_features_list if uid == user2), None)
        
        if user1_features is not None and user2_features is not None:
            squared_diff = np.square(user1_features.values - user2_features.values)
            X.append(squared_diff.flatten())
            y.append(label)

    return np.array(X), np.array(y)

user_features_gen = user_features_generator(dataset1e)
X, y = process_dataset(dataset2, user_features_gen)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_test, X_validate, y_test, y_validate = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
eer_threshold_idx_test = np.argmin(np.abs(tpr_test - (1 - fpr_test)))
fpr_test_val = fpr_test[eer_threshold_idx_test]
tpr_test_val = tpr_test[eer_threshold_idx_test]
eer_test_val = (fpr_test_val + 1 - tpr_test_val) / 2
print("FPR for Test Set:", fpr_test_val)
print("TPR for Test Set:", tpr_test_val)
print("EER for Test Set:", eer_test_val)


y_pred_validate = model.predict(X_validate)
accuracy_validate = accuracy_score(y_validate, y_pred_validate)
print("Validation Accuracy:", accuracy_validate)
print("Validation Confusion Matrix:")
print(confusion_matrix(y_validate, y_pred_validate))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

fpr_val, tpr_val, _ = roc_curve(y_validate, model.predict_proba(X_validate)[:, 1])
eer_threshold_idx_val = np.argmin(np.abs(tpr_val - (1 - fpr_val)))
fpr_val_val = fpr_val[eer_threshold_idx_val]
tpr_val_val = tpr_val[eer_threshold_idx_val]
eer_val_val = (fpr_val_val + 1 - tpr_val_val) / 2
print("FPR for Validation Set:", fpr_val_val)
print("TPR for Validation Set:", tpr_val_val)
print("EER for Validation Set:", eer_val_val)

Processing rows: 100%|██████████| 42312/42312 [01:15<00:00, 558.98it/s]


Accuracy: 0.504254112308565
Confusion Matrix:
[[1507 1146]
 [1476 1160]]
FPR for Test Set: 0.502826988315115
TPR for Test Set: 0.4973444613050076
EER for Test Set: 0.5027412635050537
Validation Accuracy: 0.5112497636604273
Validation Confusion Matrix:
[[1518 1129]
 [1456 1186]]
FPR for Validation Set: 0.49867774839440876
TPR for Validation Set: 0.5015140045420137
EER for Validation Set: 0.4985818719261975


In [21]:
dataset1f = pd.read_csv("dataset1_25000.csv")
dataset1f

,USER_ID,Thil_count,Thil_mean,Thil_median,Thil_pstdev,Thil_max,Thil_min,Thpl_count,Thpl_mean,Thpl_median,...,curl_median,curl_pstdev,curl_max,curl_min,cuhl_count,cuhl_mean,cuhl_median,cuhl_pstdev,cuhl_max,cuhl_min
0,21833_1,1.0,27889.000000,27889.0,0.000000,27889.0,27889.0,1.0,112225.000000,112225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21833_2,1.0,15129.000000,15129.0,0.000000,15129.0,15129.0,1.0,555025.000000,555025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,233209_2,3.0,3744.666667,3969.0,2135.730372,6241.0,1024.0,3.0,132320.333333,20736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39734_2,2.0,2592.500000,2592.5,1503.500000,4096.0,1089.0,2.0,23552.500000,23552.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,285679_2,2.0,9176.500000,9176.5,1432.500000,10609.0,7744.0,2.0,43840.000000,43840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42730,422325_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42731,446102_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42732,266602_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42733,40737_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
dataset1f = dataset1f.applymap(lambda x: np.log1p(x) if isinstance(x, (int, float)) else x)
mean_values = dataset1f.mean()
dataset1f.fillna(mean_values, inplace = True)

def user_features_generator(dataset):
    for user_id, user_features in dataset.groupby("USER_ID"):
        yield user_id, user_features.drop(columns=["USER_ID"])

def process_dataset(dataset2, user_features_generator):
    user_features_list = list(user_features_generator) 
    X = []
    y = []
    total_iterations = len(dataset2)

    for _, row in tqdm(dataset2.iterrows(), total=total_iterations, desc="Processing rows"):
        user1, user2, label = row["User ID #1"], row["User ID #2"], row["Label"]
        user1_features = next((features for uid, features in user_features_list if uid == user1), None)
        user2_features = next((features for uid, features in user_features_list if uid == user2), None)
        
        if user1_features is not None and user2_features is not None:
            squared_diff = np.square(user1_features.values - user2_features.values)
            X.append(squared_diff.flatten())
            y.append(label)

    return np.array(X), np.array(y)

user_features_gen = user_features_generator(dataset1f)
X, y = process_dataset(dataset2, user_features_gen)

Processing rows: 100%|██████████| 42312/42312 [01:15<00:00, 561.17it/s]


In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
combined2 = scaler.fit_transform(X)
columns = [f"Feature_{i}" for i in range(combined2.shape[1])]
combined2_df_s = pd.DataFrame(combined2, columns=columns)
X = combined2_df_s

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_test, X_validate, y_test, y_validate = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
eer_threshold_idx_test = np.argmin(np.abs(tpr_test - (1 - fpr_test)))
fpr_test_val = fpr_test[eer_threshold_idx_test]
tpr_test_val = tpr_test[eer_threshold_idx_test]
eer_test_val = (fpr_test_val + 1 - tpr_test_val) / 2
print("FPR for Test Set:", fpr_test_val)
print("TPR for Test Set:", tpr_test_val)
print("EER for Test Set:", eer_test_val)


y_pred_validate = model.predict(X_validate)
accuracy_validate = accuracy_score(y_validate, y_pred_validate)
print("Validation Accuracy:", accuracy_validate)
print("Validation Confusion Matrix:")
print(confusion_matrix(y_validate, y_pred_validate))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

fpr_val, tpr_val, _ = roc_curve(y_validate, model.predict_proba(X_validate)[:, 1])
eer_threshold_idx_val = np.argmin(np.abs(tpr_val - (1 - fpr_val)))
fpr_val_val = fpr_val[eer_threshold_idx_val]
tpr_val_val = tpr_val[eer_threshold_idx_val]
eer_val_val = (fpr_val_val + 1 - tpr_val_val) / 2
print("FPR for Validation Set:", fpr_val_val)
print("TPR for Validation Set:", tpr_val_val)
print("EER for Validation Set:", eer_val_val)

Accuracy: 0.5046322556248818
Confusion Matrix:
[[1468 1185]
 [1435 1201]]
FPR for Test Set: 0.499434602336977
TPR for Test Set: 0.5003793626707133
EER for Test Set: 0.49952761983313193
Validation Accuracy: 0.5150311968235961
Validation Confusion Matrix:
[[1507 1140]
 [1425 1217]]
FPR for Validation Set: 0.49301095579901777
TPR for Validation Set: 0.5068130204390613
EER for Validation Set: 0.4930989676799782


In [24]:
dataset1g = pd.read_csv("dataset1_25000.csv")
dataset1g

,USER_ID,Thil_count,Thil_mean,Thil_median,Thil_pstdev,Thil_max,Thil_min,Thpl_count,Thpl_mean,Thpl_median,...,curl_median,curl_pstdev,curl_max,curl_min,cuhl_count,cuhl_mean,cuhl_median,cuhl_pstdev,cuhl_max,cuhl_min
0,21833_1,1.0,27889.000000,27889.0,0.000000,27889.0,27889.0,1.0,112225.000000,112225.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21833_2,1.0,15129.000000,15129.0,0.000000,15129.0,15129.0,1.0,555025.000000,555025.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,233209_2,3.0,3744.666667,3969.0,2135.730372,6241.0,1024.0,3.0,132320.333333,20736.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39734_2,2.0,2592.500000,2592.5,1503.500000,4096.0,1089.0,2.0,23552.500000,23552.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,285679_2,2.0,9176.500000,9176.5,1432.500000,10609.0,7744.0,2.0,43840.000000,43840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42730,422325_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42731,446102_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42732,266602_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42733,40737_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
dataset1g = dataset1g.applymap(lambda x: np.log1p(x) if isinstance(x, (int, float)) else x)
mean_values = dataset1g.mean()
dataset1g.fillna(mean_values, inplace = True)

def user_features_generator(dataset):
    for user_id, user_features in dataset.groupby("USER_ID"):
        yield user_id, user_features.drop(columns=["USER_ID"])

def process_dataset(dataset2, user_features_generator):
    user_features_list = list(user_features_generator) 
    X = []
    y = []
    total_iterations = len(dataset2)

    for _, row in tqdm(dataset2.iterrows(), total=total_iterations, desc="Processing rows"):
        user1, user2, label = row["User ID #1"], row["User ID #2"], row["Label"]
        user1_features = next((features for uid, features in user_features_list if uid == user1), None)
        user2_features = next((features for uid, features in user_features_list if uid == user2), None)
        
        if user1_features is not None and user2_features is not None:
            squared_diff = np.square(user1_features.values - user2_features.values)
            X.append(squared_diff.flatten())
            y.append(label)

    return np.array(X), np.array(y)

user_features_gen = user_features_generator(dataset1g)
X, y = process_dataset(dataset2, user_features_gen)

Processing rows: 100%|██████████| 42312/42312 [01:16<00:00, 553.05it/s]


In [26]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
combined2 = scaler.fit_transform(X)
columns = [f"Feature_{i}" for i in range(combined2.shape[1])]
combined2_df_s = pd.DataFrame(combined2, columns=columns)
X = combined2_df_s

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42)
X_test, X_validate, y_test, y_validate = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
eer_threshold_idx_test = np.argmin(np.abs(tpr_test - (1 - fpr_test)))
fpr_test_val = fpr_test[eer_threshold_idx_test]
tpr_test_val = tpr_test[eer_threshold_idx_test]
eer_test_val = (fpr_test_val + 1 - tpr_test_val) / 2
print("FPR for Test Set:", fpr_test_val)
print("TPR for Test Set:", tpr_test_val)
print("EER for Test Set:", eer_test_val)


y_pred_validate = model.predict(X_validate)
accuracy_validate = accuracy_score(y_validate, y_pred_validate)
print("Validation Accuracy:", accuracy_validate)
print("Validation Confusion Matrix:")
print(confusion_matrix(y_validate, y_pred_validate))

fpr_test, tpr_test, _ = roc_curve(y_test, model.predict_proba(X_test)[:, 1])

fpr_val, tpr_val, _ = roc_curve(y_validate, model.predict_proba(X_validate)[:, 1])
eer_threshold_idx_val = np.argmin(np.abs(tpr_val - (1 - fpr_val)))
fpr_val_val = fpr_val[eer_threshold_idx_val]
tpr_val_val = tpr_val[eer_threshold_idx_val]
eer_val_val = (fpr_val_val + 1 - tpr_val_val) / 2
print("FPR for Validation Set:", fpr_val_val)
print("TPR for Validation Set:", tpr_val_val)
print("EER for Validation Set:", eer_val_val)

Accuracy: 0.504254112308565
Confusion Matrix:
[[1507 1146]
 [1476 1160]]
FPR for Test Set: 0.502826988315115
TPR for Test Set: 0.4973444613050076
EER for Test Set: 0.5027412635050537
Validation Accuracy: 0.5112497636604273
Validation Confusion Matrix:
[[1518 1129]
 [1456 1186]]
FPR for Validation Set: 0.49867774839440876
TPR for Validation Set: 0.5015140045420137
EER for Validation Set: 0.4985818719261975
